## Prudential Life Insurance Dataset - Preliminary Data Cleaning 

As discussed in the previous notebook, there are many features (many which are nulls) that could be converted to categorical features. We will do this now to ensure that our algorithm does not run into any errors.

We will also turn this problem from an multi-class prediction problem (Response 1-8) into a single class prediction problem (Response 1-7 together, 8 alone) since it appears that response 8 is the most distinct from the other classes.

In [3]:
# Common libraries 
import os
from tqdm.notebook import tqdm 

# Data analytic/viz libraries s
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
train = pd.read_csv('train.csv', index_col=0)

In [5]:
response = train['Response']

Easier to break this up into categories and then merge them together

### Product Features

In [6]:
product_features = train.iloc[:, :7]
product_features.head()

,Product_Info_1,Product_Info_2,Product_Info_3,Product_Info_4,Product_Info_5,Product_Info_6,Product_Info_7
Id,,,,,,,
2,1,D3,10,0.076923,2,1,1
5,1,A1,26,0.076923,2,3,1
6,1,E1,26,0.076923,2,3,1
7,1,D4,10,0.487179,2,3,1
8,1,D2,26,0.230769,2,3,1


In [7]:
product_categorical = [f'Product_Info_{i}' for i in [1,2,5,6,7]]
product_continuous = [col for col in product_features.columns if col not in product_categorical]

In [8]:
product_dummies = pd.get_dummies(product_features[product_categorical].astype('str'), drop_first=True)

In [9]:
product_cleaned = pd.concat([product_dummies, product_features[product_continuous]], axis=1)

In [10]:
product_cleaned.isnull().sum()

Product_Info_1_2     0
Product_Info_2_A2    0
Product_Info_2_A3    0
Product_Info_2_A4    0
Product_Info_2_A5    0
Product_Info_2_A6    0
Product_Info_2_A7    0
Product_Info_2_A8    0
Product_Info_2_B1    0
Product_Info_2_B2    0
Product_Info_2_C1    0
Product_Info_2_C2    0
Product_Info_2_C3    0
Product_Info_2_C4    0
Product_Info_2_D1    0
Product_Info_2_D2    0
Product_Info_2_D3    0
Product_Info_2_D4    0
Product_Info_2_E1    0
Product_Info_5_3     0
Product_Info_6_3     0
Product_Info_7_2     0
Product_Info_7_3     0
Product_Info_3       0
Product_Info_4       0
dtype: int64

### Physical Features (already cleaned)

In [11]:
physical_features = train.iloc[:, 7:11]
physical_features.head()

,Ins_Age,Ht,Wt,BMI
Id,,,,
2,0.641791,0.581818,0.148536,0.323008
5,0.059701,0.600000,0.131799,0.272288
6,0.029851,0.745455,0.288703,0.428780
7,0.164179,0.672727,0.205021,0.352438
8,0.417910,0.654545,0.234310,0.424046


In [12]:
physical_cleaned = physical_features

### Employment Features

In [13]:
employment_features = train.iloc[:, 11:17].copy()
employment_features.head()

,Employment_Info_1,Employment_Info_2,Employment_Info_3,Employment_Info_4,Employment_Info_5,Employment_Info_6
Id,,,,,,
2,0.028,12,1,0.0,3,NaN
5,0.000,1,3,0.0,2,0.0018
6,0.030,9,1,0.0,2,0.0300
7,0.042,9,1,0.0,3,0.2000
8,0.027,9,1,0.0,2,0.0500


In [14]:
employment_features.isnull().sum()

Employment_Info_1       19
Employment_Info_2        0
Employment_Info_3        0
Employment_Info_4     6779
Employment_Info_5        0
Employment_Info_6    10854
dtype: int64

In [15]:
# Only 19 missing values.. Impute with mode (0)
employment_features['Employment_Info_1'].fillna(0, inplace=True)

# This feature has generally low variability, so imputing with mean should be okay
employment_features['Employment_Info_4'].fillna(employment_features['Employment_Info_4'].mean(), inplace=True)

# This feature has low variance within each of its distributions
# Impute with mode (0)
employment_features['Employment_Info_6'].fillna(0, inplace=True)

In [16]:
employment_features_categorical = [f'Employment_Info_{i}' for i in [3,5]]
employment_features_continuous = [col for col in employment_features.columns if col not in employment_features_categorical]

In [17]:
employment_dummies = pd.get_dummies(employment_features[employment_features_categorical].astype('str'), drop_first=True)

In [18]:
employment_cleaned = pd.concat([employment_dummies, employment_features[employment_features_continuous]], axis=1)

In [19]:
employment_cleaned.isnull().sum()

Employment_Info_3_3    0
Employment_Info_5_3    0
Employment_Info_1      0
Employment_Info_2      0
Employment_Info_4      0
Employment_Info_6      0
dtype: int64

### Insured Features

In [20]:
insured_features = train.iloc[:, 17:24]
insured_features.head()

,InsuredInfo_1,InsuredInfo_2,InsuredInfo_3,InsuredInfo_4,InsuredInfo_5,InsuredInfo_6,InsuredInfo_7
Id,,,,,,,
2,1,2,6,3,1,2,1
5,1,2,6,3,1,2,1
6,1,2,8,3,1,1,1
7,2,2,8,3,1,2,1
8,1,2,6,3,1,2,1


In [21]:
insured_features.isnull().sum()

InsuredInfo_1    0
InsuredInfo_2    0
InsuredInfo_3    0
InsuredInfo_4    0
InsuredInfo_5    0
InsuredInfo_6    0
InsuredInfo_7    0
dtype: int64

In [22]:
insured_features_categorical = [f'InsuredInfo_{i}' for i in [1,2,4,5,6,7]]
insured_features_continuous = ['InsuredInfo_3']

In [23]:
insured_dummies = pd.get_dummies(insured_features[insured_features_categorical].astype('str'), drop_first=True)

In [24]:
insured_clean = pd.concat([insured_dummies, insured_features[insured_features_continuous]], axis=1)

In [25]:
insured_clean.isnull().sum()

InsuredInfo_1_2    0
InsuredInfo_1_3    0
InsuredInfo_2_3    0
InsuredInfo_4_3    0
InsuredInfo_5_3    0
InsuredInfo_6_2    0
InsuredInfo_7_3    0
InsuredInfo_3      0
dtype: int64

### Insured History Features

In [26]:
insured_hist_features = train.iloc[:, 24: 32].copy()
insured_hist_features.head()

,Insurance_History_1,Insurance_History_2,Insurance_History_3,Insurance_History_4,Insurance_History_5,Insurance_History_7,Insurance_History_8,Insurance_History_9
Id,,,,,,,,
2,1,1,3,1,0.000667,1,1,2
5,2,1,3,1,0.000133,1,3,2
6,2,1,1,3,NaN,3,2,3
7,2,1,1,3,NaN,3,2,3
8,2,1,1,3,NaN,3,2,3


In [27]:
ins_hist_features_categorical = [f'Insurance_History_{i}' for i in [1,2,3,4,7,8,9]]
insured_features_continuous = ['Insurance_History_5']

In [28]:
# Fill the nulls with 0 (mode)
insured_hist_features['Insurance_History_5'].fillna(0, inplace=True)

In [29]:
insured_hist_dummies = pd.get_dummies(insured_hist_features[ins_hist_features_categorical].astype('str'), drop_first=True)

In [30]:
insured_hist_cleaned = pd.concat([insured_hist_dummies, insured_hist_features[insured_features_continuous]], axis=1)

In [31]:
insured_hist_cleaned.head()

,Insurance_History_1_2,Insurance_History_2_2,Insurance_History_2_3,Insurance_History_3_2,Insurance_History_3_3,Insurance_History_4_2,Insurance_History_4_3,Insurance_History_7_2,Insurance_History_7_3,Insurance_History_8_2,Insurance_History_8_3,Insurance_History_9_2,Insurance_History_9_3,Insurance_History_5
Id,,,,,,,,,,,,,,
2,0,0,0,0,1,0,0,0,0,0,0,1,0,0.000667
5,1,0,0,0,1,0,0,0,0,0,1,1,0,0.000133
6,1,0,0,0,0,0,1,0,1,1,0,0,1,0.000000
7,1,0,0,0,0,0,1,0,1,1,0,0,1,0.000000
8,1,0,0,0,0,0,1,0,1,1,0,0,1,0.000000


In [32]:
insured_hist_cleaned.isnull().sum()

Insurance_History_1_2    0
Insurance_History_2_2    0
Insurance_History_2_3    0
Insurance_History_3_2    0
Insurance_History_3_3    0
Insurance_History_4_2    0
Insurance_History_4_3    0
Insurance_History_7_2    0
Insurance_History_7_3    0
Insurance_History_8_2    0
Insurance_History_8_3    0
Insurance_History_9_2    0
Insurance_History_9_3    0
Insurance_History_5      0
dtype: int64

### Family History Features

In [33]:
famhist_features = train.iloc[:, 32:37].copy()
famhist_features.head()

,Family_Hist_1,Family_Hist_2,Family_Hist_3,Family_Hist_4,Family_Hist_5
Id,,,,,
2,2,NaN,0.598039,NaN,0.526786
5,2,0.188406,NaN,0.084507,NaN
6,3,0.304348,NaN,0.225352,NaN
7,3,0.420290,NaN,0.352113,NaN
8,2,0.463768,NaN,0.408451,NaN


In [34]:
famhist_features.isnull().sum()

Family_Hist_1        0
Family_Hist_2    28656
Family_Hist_3    34241
Family_Hist_4    19184
Family_Hist_5    41811
dtype: int64

From the previous notebook, we discussed splitting some of these columns into isnull and not null, since there appeared to be greater separation from that.

In [35]:
# Split into nulls and not nulls
famhist_features['Family_Hist_2'] = famhist_features['Family_Hist_2'].notnull().astype('int')
famhist_features['Family_Hist_3'] = famhist_features['Family_Hist_3'].notnull().astype('int')
famhist_features['Family_Hist_4'] = famhist_features['Family_Hist_4'].notnull().astype('int')
famhist_features['Family_Hist_5'] = famhist_features['Family_Hist_5'].notnull().astype('int')

In [36]:
# They are all categorical features
famhist_cleaned = pd.get_dummies(famhist_features.astype('str'), drop_first=True)

### Medical History 

In [37]:
medhist_features = train.iloc[:, 37: 78].copy()
medhist_features.head()

,Medical_History_1,Medical_History_2,Medical_History_3,Medical_History_4,Medical_History_5,Medical_History_6,Medical_History_7,Medical_History_8,Medical_History_9,Medical_History_10,...,Medical_History_32,Medical_History_33,Medical_History_34,Medical_History_35,Medical_History_36,Medical_History_37,Medical_History_38,Medical_History_39,Medical_History_40,Medical_History_41
Id,,,,,,,,,,,,,,,,,,,,,
2,4.0,112,2,1,1,3,2,2,1,NaN,...,NaN,1,3,1,2,2,1,3,3,3
5,5.0,412,2,1,1,3,2,2,1,NaN,...,NaN,3,1,1,2,2,1,3,3,1
6,10.0,3,2,2,1,3,2,2,2,NaN,...,NaN,3,3,1,3,2,1,3,3,1
7,0.0,350,2,2,1,3,2,2,2,NaN,...,NaN,3,3,1,2,2,1,3,3,1
8,NaN,162,2,2,1,3,2,2,2,NaN,...,NaN,3,3,1,3,2,1,3,3,1


In [38]:
medhist_features.isnull().sum()/medhist_features.shape[0]

Medical_History_1     0.149694
Medical_History_2     0.000000
Medical_History_3     0.000000
Medical_History_4     0.000000
Medical_History_5     0.000000
Medical_History_6     0.000000
Medical_History_7     0.000000
Medical_History_8     0.000000
Medical_History_9     0.000000
Medical_History_10    0.990620
Medical_History_11    0.000000
Medical_History_12    0.000000
Medical_History_13    0.000000
Medical_History_14    0.000000
Medical_History_15    0.751015
Medical_History_16    0.000000
Medical_History_17    0.000000
Medical_History_18    0.000000
Medical_History_19    0.000000
Medical_History_20    0.000000
Medical_History_21    0.000000
Medical_History_22    0.000000
Medical_History_23    0.000000
Medical_History_24    0.935990
Medical_History_25    0.000000
Medical_History_26    0.000000
Medical_History_27    0.000000
Medical_History_28    0.000000
Medical_History_29    0.000000
Medical_History_30    0.000000
Medical_History_31    0.000000
Medical_History_32    0.981358
Medical_

In [39]:
# Impute with mode 
medhist_features['Medical_History_1'].fillna(0, inplace=True)

# Drop column, 99% null
medhist_features.drop(columns=['Medical_History_10'], inplace=True)

# Transform into into 0 (null), 1 (lower half), 2 (upper half)
medhist_features['Medical_History_15'] = pd.qcut(medhist_features['Medical_History_15'], 2, labels=[1,2]).cat.codes.astype('str')

# Drop, 93% nulls with no clear separation
medhist_features.drop(columns=['Medical_History_24'], inplace=True)

# Drop, 98% nulls with no clear separation 
medhist_features.drop(columns=['Medical_History_32'], inplace=True)

Resulting dataframe is all categorical with exception of Medical_History_1, Medical_History_2

In [40]:
medhist_features_continuous = ['Medical_History_1', 'Medical_History_2']
medhist_features_categorical = [col for col in medhist_features.columns if col not in medhist_features_continuous]

In [41]:
medhist_dummies = pd.get_dummies(medhist_features[medhist_features_categorical].astype('str'), drop_first=True)

In [42]:
medhist_dummies

,Medical_History_3_2,Medical_History_3_3,Medical_History_4_2,Medical_History_5_2,Medical_History_5_3,Medical_History_6_2,Medical_History_6_3,Medical_History_7_2,Medical_History_7_3,Medical_History_8_2,...,Medical_History_36_3,Medical_History_37_2,Medical_History_37_3,Medical_History_38_2,Medical_History_39_2,Medical_History_39_3,Medical_History_40_2,Medical_History_40_3,Medical_History_41_2,Medical_History_41_3
Id,,,,,,,,,,,,,,,,,,,,,
2,1,0,0,0,0,0,1,1,0,1,...,0,1,0,0,0,1,0,1,0,1
5,1,0,0,0,0,0,1,1,0,1,...,0,1,0,0,0,1,0,1,0,0
6,1,0,1,0,0,0,1,1,0,1,...,1,1,0,0,0,1,0,1,0,0
7,1,0,1,0,0,0,1,1,0,1,...,0,1,0,0,0,1,0,1,0,0
8,1,0,1,0,0,0,1,1,0,1,...,1,1,0,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79142,1,0,0,0,0,0,1,1,0,1,...,0,1,0,0,0,1,0,1,0,1
79143,1,0,1,0,0,0,1,1,0,1,...,1,1,0,0,0,1,0,1,0,0
79144,1,0,1,0,0,0,1,1,0,1,...,1,1,0,0,0,1,0,1,0,0


In [43]:
medhist_cleaned = pd.concat([medhist_dummies, medhist_features[medhist_features_continuous]], axis=1)

### Medkey Features

In [47]:
medkey_features = train.iloc[:, 79:-1].copy()
medkey_features.head()

,Medical_Keyword_2,Medical_Keyword_3,Medical_Keyword_4,Medical_Keyword_5,Medical_Keyword_6,Medical_Keyword_7,Medical_Keyword_8,Medical_Keyword_9,Medical_Keyword_10,Medical_Keyword_11,...,Medical_Keyword_39,Medical_Keyword_40,Medical_Keyword_41,Medical_Keyword_42,Medical_Keyword_43,Medical_Keyword_44,Medical_Keyword_45,Medical_Keyword_46,Medical_Keyword_47,Medical_Keyword_48
Id,,,,,,,,,,,,,,,,,,,,,
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [49]:
# Add a column that represents the number of medkey features 
medkey_features['Medical_Keyword_Count'] = medkey_features.sum(axis=1)

They are all categorical features, with the exception of the last columns

In [50]:
medkey_cleaned = medkey_features

## Getting the Cleaned Dataframe

Note: Include all data manipulations in the future for test set/new data.

In [51]:
# Concat all cleaned dfs into a single df for future use
all_dfs = [physical_cleaned, employment_cleaned, insured_clean, insured_hist_cleaned, famhist_cleaned, medhist_cleaned,
          medkey_cleaned, response]

clean_train = pd.concat(all_dfs, axis=1)

In [52]:
clean_train.to_csv('cleaned_train.csv')